Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 1

## Assignment
- [ ] Do train/validate/test split with the Tanzania Waterpumps data.
- [ ] Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features. (For example, [what other columns have zeros and shouldn't?](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values) What other columns are duplicates, or nearly duplicates? Can you extract the year from date_recorded? Can you engineer new features, such as the number of years from waterpump construction to waterpump inspection?)
- [ ] Select features. Use a scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your feature importances.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

### Reading

- A Visual Introduction to Machine Learning
  - [Part 1: A Decision Tree](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
  - [Part 2: Bias and Variance](http://www.r2d3.us/visual-intro-to-machine-learning-part-2/)
- [Decision Trees: Advantages & Disadvantages](https://christophm.github.io/interpretable-ml-book/tree.html#advantages-2)
- [How a Russian mathematician constructed a decision tree — by hand — to solve a medical problem](http://fastml.com/how-a-russian-mathematician-constructed-a-decision-tree-by-hand-to-solve-a-medical-problem/)
- [How decision trees work](https://brohrer.github.io/how_decision_trees_work.html)
- [Let’s Write a Decision Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU) — _Don’t worry about understanding the code, just get introduced to the concepts. This 10 minute video has excellent diagrams and explanations._
- [Random Forests for Complete Beginners: The definitive guide to Random Forests and Decision Trees](https://victorzhou.com/blog/intro-to-random-forests/)


### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Try other [scikit-learn imputers](https://scikit-learn.org/stable/modules/impute.html).
- [ ] Try other [scikit-learn scalers](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Make exploratory visualizations and share on Slack.


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from a previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```



In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.impute import SimpleImputer

In [3]:
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')
train,val = train_test_split(train,random_state = 32,stratify=train['status_group'],test_size=0.20)
train.shape, test.shape

FileNotFoundError: ignored

In [0]:
#After doing exhaustive study of data both graphically and by basic functions , the following are the observations
#Not relevant or too many null vaues funder, installer,scheme_name,num_private,scheme_name
#High cardinality wpt_name,subvillage,lga,ward
#remove region code as region is considered
#remove scheme_management as management is considered
#constant value recorded_by
#keeping extraction_type and removing extraction_type_group and extraction_type_class
#payment_type and payment are same
#removing waterpoint_type_group as it can be replaced with waterpoint_type
#keeping water_quality as same as quality_group
#quantity_group and quantity are same
#source and source_type are same 

drop_columns = ['funder','installer','scheme_name','scheme_name','scheme_name','wpt_name',
                'subvillage','lga','ward','recorded_by','extraction_type_group','extraction_type_class',
                'payment_type','waterpoint_type_group','quality_group','quantity_group','source_type','id']
#To avoid copy setting warning, creating a copy of the datasets
train = train.copy()
val = val.copy()

train.drop(columns=drop_columns,inplace=True)
val.drop(columns=drop_columns,inplace=True)
test.drop(columns=drop_columns,inplace=True)

In [0]:

train['latitude'] = train['latitude'].replace(-2e-08, 0)
val['latitude'] = val['latitude'].replace(-2e-08, 0)
test['latitude'] = test['latitude'].replace(-2e-08, 0)

In [0]:
nan_columns = ['latitude','longitude']

In [0]:
def make_zero_nan(data,col):
  X = data.copy()
  X[col]=X[col].replace(0,np.NaN)
  return X

In [0]:
for col in nan_columns:
  for df in [train,val,test]:
    df = make_zero_nan(df,col)

In [112]:
train[['latitude','longitude']].describe()

,latitude,longitude
count,47520.000000,47520.000000
mean,-5.708551,34.100222
std,2.942088,6.516114
min,-11.649440,0.000000
25%,-8.552568,33.100053
50%,-5.017987,34.909100
75%,-3.327596,37.180879
max,0.000000,40.344301


In [1]:
target = 'status_group'
features = train.columns.drop(target)
Y_train = train[target]
Y_val = val[target]
X_train = train[features]
X_val = val[features]
#making a pipeline for model testing

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    DecisionTreeClassifier(random_state=32,min_samples_leaf=20)
)
pipeline.fit(X_train,Y_train)
pred_train = pipeline.predict(X_train)
y_pred = pipeline.predict(X_val)
print("Training Score:",accuracy_score(Y_train,pred_train))
print("Val Score:",accuracy_score(Y_val,y_pred))


NameError: ignored

In [0]:
model = pipeline.named_steps['decisiontreeclassifier']
encoder = pipeline.named_steps['onehotencoder']

In [48]:
encoder.transform(X_val).columns.to_list()

['amount_tsh',
 'date_recorded_2013-03-15',
 'date_recorded_2011-03-05',
 'date_recorded_2011-04-01',
 'date_recorded_2013-03-04',
 'date_recorded_2012-10-29',
 'date_recorded_2012-10-24',
 'date_recorded_2011-03-22',
 'date_recorded_2013-01-20',
 'date_recorded_2011-03-19',
 'date_recorded_2011-07-26',
 'date_recorded_2012-10-31',
 'date_recorded_2011-07-21',
 'date_recorded_2013-02-13',
 'date_recorded_2013-03-23',
 'date_recorded_2011-08-08',
 'date_recorded_2013-02-09',
 'date_recorded_2011-02-26',
 'date_recorded_2013-02-27',
 'date_recorded_2013-02-25',
 'date_recorded_2013-02-18',
 'date_recorded_2013-11-03',
 'date_recorded_2013-03-13',
 'date_recorded_2013-03-19',
 'date_recorded_2012-10-18',
 'date_recorded_2013-02-26',
 'date_recorded_2013-09-03',
 'date_recorded_2013-03-16',
 'date_recorded_2011-03-14',
 'date_recorded_2011-01-04',
 'date_recorded_2013-03-18',
 'date_recorded_2011-02-23',
 'date_recorded_2013-03-17',
 'date_recorded_2011-07-31',
 'date_recorded_2013-01-31',

In [0]:
features = pd.DataFrame({'columns':encoder.transform(X_val).columns.to_list(),'importance':model.feature_importances_})

In [70]:
#plotting the importance

features.sort_values('importance',ascending=False)[:20]


,columns,importance
453,quantity_dry,0.251275
474,waterpoint_type_other,0.131646
347,longitude,0.112980
348,latitude,0.084768
0,amount_tsh,0.044831
402,construction_year,0.043088
473,waterpoint_type_communal standpipe multiple,0.031041
346,gps_height,0.029183
381,district_code,0.023249
382,population,0.022323


In [36]:
features.head()

0    Index(['amount_tsh', 'date_recorded_2013-03-15...
1    [0.044831299635566146, 5.743392291606526e-05, ...
dtype: object